In [20]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import datetime
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")

In [21]:
buildings_data_file_path = 'data/preprocessed/buildings_data.geojson'
hex_grid_path = 'data/preprocessed/hex_grid.geojson'

In [22]:
def parse_hex_grid():
    gdf = gpd.read_file(hex_grid_path)
    return gdf

In [23]:
hex_grid_gdf = parse_hex_grid()
estimated_utm_crs = hex_grid_gdf.estimate_utm_crs()
hex_grid_gdf_projected = hex_grid_gdf.to_crs(estimated_utm_crs)

In [24]:
def parse_buildings_data():
    gdf = gpd.read_file(buildings_data_file_path)
    estimated_utm_crs = gdf.estimate_utm_crs()
    gdf_projected = gdf.to_crs(estimated_utm_crs)
    return gdf_projected

In [25]:
buidlings_gdf = parse_buildings_data()

In [26]:
def compute_building_counts(hex_grid_gdf, buidlings_gdf):
    joined_gdf = gpd.sjoin(buidlings_gdf, hex_grid_gdf, how="left", predicate="within")
    building_counts = joined_gdf.groupby('index_right').size().reset_index(name='Building Count')

    hex_grid_gdf = hex_grid_gdf.reset_index().merge(building_counts, left_on='index', right_on='index_right', how='left').set_index('index')
    hex_grid_gdf['Building Count'] = hex_grid_gdf['Building Count'].fillna(0).astype(int)  # Handle regions with no points
    return hex_grid_gdf

In [27]:
hex_grid_gdf_projected = compute_building_counts(hex_grid_gdf_projected, buidlings_gdf)
hex_grid_gdf['Building Count'] = hex_grid_gdf_projected['Building Count']

In [28]:
hex_grid_gdf.to_file('data/preprocessed/hex_grid.geojson', driver="GeoJSON")